In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D


print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.13.0


In [3]:
WORK_DIR = './Alzheimers-OASIS'

CLASSES = [ 'Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia',]

IMG_SIZE = 248
IMAGE_SIZE = [248, 248]
DIM = (IMG_SIZE, IMG_SIZE)

In [4]:
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = work_dr.flow_from_directory(directory=WORK_DIR, target_size=DIM, batch_size=9000, shuffle=False)

Found 86437 images belonging to 4 classes.


In [5]:
train_data, train_labels = train_data_gen.next()

In [6]:
print(train_data.shape, train_labels.shape)

(9000, 248, 248, 3) (9000, 4)


In [7]:
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [8]:
vgg19_model = VGG19(input_shape=(248, 248, 3), include_top=False, weights="imagenet")

In [9]:
for layer in vgg19_model.layers:
    layer.trainable=False

In [11]:
custom_vgg19_model = Sequential([
        vgg19_model,
        Dropout(0.5),
        GlobalAveragePooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(4, activation='softmax')        
    ], name = "vgg19model")

In [12]:
import keras.backend as K
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [13]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]

# CALLBACKS = rop_callback
    
custom_vgg19_model.compile(optimizer='rmsprop',
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

custom_vgg19_model.summary()

Model: "vgg19model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 dropout_5 (Dropout)         (None, 7, 7, 512)         0         
                                                                 
 global_average_pooling2d_1  (None, 512)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 batch_normalization_5 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dense_5 (Dense)             (None, 512)               2

In [14]:
EPOCHS = 3

history = custom_vgg19_model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=EPOCHS,steps_per_epoch=128)

Epoch 1/3
128/128 [==============================] - 1665s 13s/step - loss: 1.3818 - accuracy: 0.7516 - precision: 0.5048 - recall: 0.3306 - auc: 0.6982 - f1_score: 0.3979 - val_loss: 0.9753 - val_accuracy: 0.8132 - val_precision: 0.6753 - val_recall: 0.4868 - val_auc: 0.8824 - val_f1_score: 0.5647
Epoch 2/3
128/128 [==============================] - 1701s 13s/step - loss: 0.8381 - accuracy: 0.8451 - precision: 0.7261 - recall: 0.6111 - auc: 0.8807 - f1_score: 0.6621 - val_loss: 0.8096 - val_accuracy: 0.7899 - val_precision: 0.5819 - val_recall: 0.5674 - val_auc: 0.9033 - val_f1_score: 0.5746
Epoch 3/3
128/128 [==============================] - 1623s 13s/step - loss: 0.6370 - accuracy: 0.8797 - precision: 0.7765 - recall: 0.7286 - auc: 0.9288 - f1_score: 0.7517 - val_loss: 0.6053 - val_accuracy: 0.8757 - val_precision: 0.7710 - val_recall: 0.7153 - val_auc: 0.9449 - val_f1_score: 0.7414


In [15]:
#Evaluating the model on the data

#train_scores = model.evaluate(train_data, train_labels)
#val_scores = model.evaluate(val_data, val_labels)
test_scores = custom_vgg19_model.evaluate(test_data, test_labels)

#print("Training Accuracy: %.2f%%"%(train_scores[1] * 100))
#print("Validation Accuracy: %.2f%%"%(val_scores[1] * 100))
print("Testing Accuracy: %.2f%%"%(test_scores[1] * 100))

57/57 [==============================] - 407s 7s/step - loss: 0.6112 - accuracy: 0.8696 - precision: 0.7552 - recall: 0.7078 - auc: 0.9437 - f1_score: 0.7291
Testing Accuracy: 86.96%


In [16]:
pred_labels = custom_vgg19_model.predict(test_data)

57/57 [==============================] - 391s 7s/step


In [17]:
#Print the classification report of the tested data

#Since the labels are softmax arrays, we need to roundoff to have it in the form of 0s and 1s,
#similar to the test_labels
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

print(classification_report(test_labels, pred_labels, target_names=CLASSES))

                    precision    recall  f1-score   support

     Mild Dementia       0.67      0.99      0.80       979
 Moderate Dementia       0.00      0.00      0.00        97
      Non Demented       0.98      0.48      0.64       724
Very mild Dementia       0.00      0.00      0.00         0

         micro avg       0.73      0.73      0.73      1800
         macro avg       0.41      0.37      0.36      1800
      weighted avg       0.76      0.73      0.69      1800
       samples avg       0.73      0.73      0.73      1800



c:\logs\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\logs\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
